In [15]:
from pandas_datareader import data as web
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [16]:
start = '1990-01-31'
end = '2021-12-31'

In [17]:
leadingIndicator = pd.read_csv('./leadingIndicator.csv')  # monthly 지표
leadingIndicator = leadingIndicator.set_index('DATE')
leadingIndicator.index = pd.to_datetime(leadingIndicator.index)

vix = web.DataReader('VIXCLS', 'fred', start, end)
vix = vix.resample('M').last()

gdp_growth = web.DataReader('NAEXKP04USQ657S', 'fred', start, end)  # 분기별 실질 GDP 성장률
gdp_growth_monthly = gdp_growth.resample('M').ffill(limit=3)  # 분기 데이터를 월별로 전달

In [18]:
# indicator & vix & gdp 합치기
table = pd.concat([leadingIndicator, vix, gdp_growth_monthly], axis=1, join='inner')
table.columns = ['leadingIndicator', 'vix', 'gdp']
table = table['2003-01-01':]
table

,leadingIndicator,vix,gdp
DATE,,,
2003-01-31,98.024932,31.17,0.732600
2003-02-28,97.994586,29.63,0.732600
2003-03-31,98.026892,29.15,0.732600
2003-04-30,98.146137,21.21,1.949934
2003-05-31,98.346977,19.47,1.949934
...,...,...,...
2021-06-30,101.172892,15.83,0.921680
2021-07-31,101.129863,18.24,-0.563895
2021-08-31,101.067557,16.48,-0.563895


In [19]:
# VIX

# 데이터 불러오기
data = web.DataReader('VIXCLS', 'fred', start='1999-01-01', end='2022-12-31')
data = data.ffill()

# 이동평균 계산
data['MA_240'] = data['VIXCLS'].rolling(window=240).mean()
data['Trend'] = '상승'
data.loc[data['VIXCLS'] < data['MA_240'], 'Trend'] = '하락'

# 월별 데이터로 리샘플링 및 마지막 날짜 선택
data = data.resample('M').last()['2000-01-31':]

# 상승 및 하락에 따라 데이터 분리
df_up = data[data['Trend'] == '상승'].index
df_down = data[data['Trend'] == '하락'].index

# 리스트
df_up = (data[data['Trend'] == '상승'].index + pd.DateOffset(months=1)).strftime('%Y-%m').tolist()
df_down = (data[data['Trend'] == '하락'].index + pd.DateOffset(months=1)).strftime('%Y-%m').tolist()

In [20]:
# leading indicator

# 데이터 불러오기
data_eco = web.DataReader('USALOLITONOSTSAM', 'fred', start='2000-01-01', end='2022-12-31')
data_eco = data_eco.ffill()

# 이동평균 계산
data_eco['mean'] = data_eco['USALOLITONOSTSAM'].mean()
data_eco['Trend'] = '상승'
data_eco.loc[data_eco['USALOLITONOSTSAM'] < data_eco['mean'], 'Trend'] = '하락'

# 월별 데이터로 리샘플링 및 마지막 날짜 선택
data_eco = data_eco.resample('M').last()

# 상승 및 하락에 따라 데이터 분리
eco_up = data_eco[data_eco['Trend'] == '상승'].index
eco_down = data_eco[data_eco['Trend'] == '하락'].index

# 리스트
eco_up = (data_eco[data_eco['Trend'] == '상승'].index + pd.DateOffset(months=1)).strftime('%Y-%m').tolist()
eco_down = (data_eco[data_eco['Trend'] == '하락'].index + pd.DateOffset(months=1)).strftime('%Y-%m').tolist()

In [21]:
print('평균 이상인 구간 :', eco_up)
print('평균 이하인 구간 :', eco_down)
print('상승 중인 구간 :', df_up)
print('하락 중인 구간 :', df_down)

평균 이상인 구간 : ['2000-02', '2000-03', '2000-04', '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10', '2000-11', '2003-11', '2003-12', '2004-01', '2004-02', '2004-03', '2004-04', '2004-05', '2004-06', '2004-07', '2004-08', '2004-09', '2004-10', '2004-11', '2004-12', '2005-01', '2005-02', '2005-03', '2005-04', '2005-05', '2005-06', '2005-07', '2005-08', '2005-09', '2005-10', '2005-11', '2005-12', '2006-01', '2006-02', '2006-03', '2006-04', '2006-05', '2006-06', '2006-07', '2006-08', '2006-09', '2006-10', '2006-11', '2006-12', '2007-01', '2007-02', '2007-03', '2007-04', '2007-05', '2007-06', '2007-07', '2007-08', '2007-09', '2007-10', '2007-11', '2007-12', '2008-01', '2008-02', '2008-03', '2008-04', '2008-05', '2008-06', '2008-07', '2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06', '2011-07', '2012-03', '2012-04', '2012-05', '2012-06', '2012-12', '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-08', '2013-09', '2013-10

In [22]:
# regime 정의
all_period = sorted(list(set(eco_up + eco_down + df_up + df_down)))

recovery = []
expansion = []
slowdown = []
contraction = []

for i in all_period:
    if i in eco_down and i in df_down:
        # i = pd.datetime(i, format = '%Y-%m')
        recovery.append(i)
    if i in eco_up and i in df_down:
        expansion.append(i)
    if i in eco_up and i in df_up:
        slowdown.append(i)
    if i in eco_down and i in df_up:
        contraction.append(i)

In [23]:
table.index

DatetimeIndex(['2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30',
               '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31',
               '2003-09-30', '2003-10-31',
               ...
               '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31',
               '2021-09-30', '2021-10-31'],
              dtype='datetime64[ns]', name='DATE', length=226, freq='M')

In [24]:
table.index = table.index.strftime('%Y-%m')

# table에 regime 컬럼 추가
table['regime'] = 'Unknown'

# 각 국면에 따라 'regime' 값 설정
table.loc[table.index.isin(recovery), 'regime'] = 'Recovery'
table.loc[table.index.isin(expansion), 'regime'] = 'Expansion'
table.loc[table.index.isin(slowdown), 'regime'] = 'Slowdown'
table.loc[table.index.isin(contraction), 'regime'] = 'Contraction'

table

,leadingIndicator,vix,gdp,regime
DATE,,,,
2003-01,98.024932,31.17,0.732600,Contraction
2003-02,97.994586,29.63,0.732600,Contraction
2003-03,98.026892,29.15,0.732600,Contraction
2003-04,98.146137,21.21,1.949934,Recovery
2003-05,98.346977,19.47,1.949934,Recovery
...,...,...,...,...
2021-06,101.172892,15.83,0.921680,Expansion
2021-07,101.129863,18.24,-0.563895,Expansion
2021-08,101.067557,16.48,-0.563895,Expansion


In [25]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226 entries, 2003-01 to 2021-10
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   leadingIndicator  226 non-null    float64
 1   vix               226 non-null    float64
 2   gdp               226 non-null    float64
 3   regime            226 non-null    object 
dtypes: float64(3), object(1)
memory usage: 8.8+ KB
